In [3]:
import requests
from bs4 import BeautifulSoup
import re
import smtplib
from email.mime.text import MIMEText
import os

def get_genre():
    print("Choose your favorite genre:")
    print("1. Detectives")
    print("2. Fantasy")
    print("3. Science Fiction")
    choice = input("Enter the number of your choice (1/2/3): ").strip()
    
    # Updated URLs for each genre
    genres = {
        '1': 'https://www.litres.ru/search/?q=%D0%B4%D0%B5%D1%82%D0%B5%D0%BA%D1%82%D0%B8%D0%B2%D1%8B',
        '2': 'https://www.litres.ru/search/?q=%D1%84%D1%8D%D0%BD%D1%82%D1%8D%D0%B7%D0%B8',
        '3': 'https://www.litres.ru/search/?q=%D1%84%D0%B0%D0%BD%D1%82%D0%B0%D1%81%D1%82%D0%B8%D0%BA%D0%B0'
    }
    
    while choice not in genres:
        print("Invalid choice. Please try again.")
        choice = input("Enter the number of your choice (1/2/3): ").strip()
    
    return genres[choice]

def scrape_litres(url, limit=10):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data from litres.ru: {e}")
        return []
    
    soup = BeautifulSoup(response.content, 'html.parser')
    books = []
    
    # Updated CSS selector to match the search results page
    for item in soup.select('.art__name__wrapper'):
        title = item.select_one('.art__name')
        if title:
            books.append(title.get_text(strip=True))
        if len(books) == limit:
            break
    
    return books

def validate_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email)

def send_email(books, genre, email):
    subject = f"Top 10 {genre.capitalize()} Books"
    message = f"Here are the top 10 best-selling {genre} books on litres.ru:\n\n"
    message += "\n".join([f"{i+1}. {book}" for i, book in enumerate(books)])

    sender_email = os.getenv("SENDER_EMAIL")  # Use environment variable
    sender_password = os.getenv("SENDER_PASSWORD")  # Use environment variable

    msg = MIMEText(message)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = email

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, [email], msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

def main():
    # Step 1: Ask for genre
    genre_url = get_genre()
    genre_name = {
        'https://www.litres.ru/search/?q=%D0%B4%D0%B5%D1%82%D0%B5%D0%BA%D1%82%D0%B8%D0%B2%D1%8B': 'Detectives',
        'https://www.litres.ru/search/?q=%D1%84%D1%8D%D0%BD%D1%82%D1%8D%D0%B7%D0%B8': 'Fantasy',
        'https://www.litres.ru/search/?q=%D1%84%D0%B0%D0%BD%D1%82%D0%B0%D1%81%D1%82%D0%B8%D0%BA%D0%B0': 'Science Fiction'
    }[genre_url]
    
    # Step 2: Scrape litres.ru for top 10 books
    print("\nFetching the top 10 books...")
    books = scrape_litres(genre_url)
    
    if not books:
        print("No books found. Please try again later.")
        return
    
    print("\nTop 10 Books:")
    for i, book in enumerate(books, 1):
        print(f"{i}. {book}")
    
    # Step 3: Ask if the user wants to send the list via email
    send_choice = input("\nWould you like to receive this list by email? (yes/no): ").strip().lower()
    if send_choice == 'yes':
        while True:
            email = input("Enter your email address: ").strip()
            if validate_email(email):
                break
            print("Invalid email address. Please try again.")
        
        # Step 4: Send the email
        send_email(books, genre_name, email)

if __name__ == "__main__":
    main()

Choose your favorite genre:
1. Detectives
2. Fantasy
3. Science Fiction


Enter the number of your choice (1/2/3):  1



Fetching the top 10 books...
No books found. Please try again later.
